In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.metrics import r2_score
from gpytorch.mlls import ExactMarginalLogLikelihood
import gpytorch
import altair as alt
import matplotlib.pyplot as plt
import re
from scipy.stats import norm
from pathlib import Path
from datetime import datetime
import json

In [2]:
class GPDataLogger:
    def __init__(self, log_dir="logs",
                 summary_name="gp_summary_data_tempswing_Vextchem_diff_addfeatchempT_densB.csv",
                 preds_name="gp_predictions_data_tempswing_Vextchem_diff_addfeatchempT_densB.csv"):
        self.log_dir = Path(log_dir)
        self.log_dir.mkdir(parents=True, exist_ok=True)
        self.summary_path = self.log_dir / summary_name
        self.preds_path = self.log_dir / preds_name
        #self.bo_iter_path = self.log_dir / bo_iter_name
        #self.bo_scores_path = self.log_dir / bo_scores_name
        self.run_started_at = datetime.now().isoformat(timespec="seconds")

    def _append_df(self, df: pd.DataFrame, path: Path):
        header = not path.exists()
        df.to_csv(path, mode="a", header=header, index=False)

    # ----- GP Summary / Predictions (wie zuvor) -----
    def log_summary(self, *, T1, T2, p1, p2, results_df: pd.DataFrame, label="working_capacity", extra_info=None):
        from sklearn.metrics import r2_score
        y_true = results_df[label] if label in results_df.columns else pd.Series(dtype=float)
        y_pred = results_df.get(f"{label}_pred", pd.Series(dtype=float))
        r2 = float(r2_score(y_true, y_pred)) if len(y_true) and len(y_pred) else None
        abs_rel = results_df.get("abs_rel_deviation", pd.Series(dtype=float))
        med_ape = float(abs_rel.median()) if len(abs_rel) else None
        mean_ape = float(abs_rel.mean()) if len(abs_rel) else None
        over20 = int((abs_rel > 20).sum()) if len(abs_rel) else 0
        max_ape = float(abs_rel.max()) if len(abs_rel) else None

        folds = None
        if "fold" in results_df.columns:
            uniq_folds = sorted(results_df["fold"].dropna().unique().tolist())
            folds = ",".join(map(str, uniq_folds)) if uniq_folds else None

        row = {
            "run_started_at": self.run_started_at,
            "timestamp": datetime.now().isoformat(timespec="seconds"),
            "temperature_kelvin_1": T1,
            "temperature_kelvin_2": T2,
            "pressure_bar_1": p1,
            "pressure_bar_2": p2,
            "n_rows": int(len(results_df)),
            "r2": r2,
            "median_APE_pct": med_ape,
            "mean_APE_pct": mean_ape,
            "count_APE_gt_20pct": over20,
            "max_APE_pct": max_ape,
            "folds": folds,
        }
        if extra_info:
            row["extra_info_json"] = json.dumps(extra_info, ensure_ascii=False)
        self._append_df(pd.DataFrame([row]), self.summary_path)

    def log_predictions(self, *, T1, T2, p1, p2, results_df: pd.DataFrame, label="working_capacity"):
        out = results_df.copy()
        #out.insert(0, "run_started_at", self.run_started_at)
        #out.insert(1, "temperature_kelvin_1", T1)
        #out.insert(1, "temperature_kelvin_2", T2)
        #out.insert(2, "pressure_bar_1", p1)
        #out.insert(3, "pressure_bar_2", p2)
        for m in [label, f"{label}_pred", "abs_rel_deviation"]:
            if m not in out.columns:
                out[m] = pd.NA
        self._append_df(out, self.preds_path)


UTILITY

In [3]:
def append_log_row(log_path: str | Path, row: dict):
        log_path = Path(log_path)
        log_path.parent.mkdir(parents=True, exist_ok=True)
        df = pd.DataFrame([row])
        header = not log_path.exists()
        df.to_csv(log_path, mode="a", header=header, index=False)
        
def plot_gp_results(candidates, mean_np, std_np, ei_np, pi_np, iteration, label="target_value"):
    x = np.arange(len(candidates))  # Struktur-Index
    names = candidates["structure_name"].values

    fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
    plt.subplots_adjust(hspace=0.3)

    axs[0].plot(x, mean_np, color="C0", label="Predicted Mean")
    axs[0].fill_between(x, mean_np - std_np, mean_np + std_np, color="C0", alpha=0.3, label="Uncertainty (±1σ)")
    axs[0].set_ylabel("Predicted Mean")
    axs[0].set_title(f"Iteration {iteration}: GP Prediction (mean ± std)")
    axs[0].legend()
    axs[0].grid(alpha=0.3)

    axs[1].plot(x, ei_np, color="C1")
    axs[1].set_ylabel("Expected Improvement")
    axs[1].set_title("Expected Improvement over Candidates")
    axs[1].grid(alpha=0.3)

    axs[2].plot(x, pi_np, color="C2")
    axs[2].set_ylabel("Probability of Improvement")
    axs[2].set_xlabel("Candidate Structure Index")
    axs[2].set_title("Probability of Improvement")
    axs[2].grid(alpha=0.3)

    # Optional: Struktur-Namen als xticks (nur alle paar, sonst zu viele)
    if len(names) <= 30:
        axs[2].set_xticks(x)
        axs[2].set_xticklabels(names, rotation=90)
    else:
        step = max(1, len(names)//30)
        axs[2].set_xticks(x[::step])
        axs[2].set_xticklabels(names[::step], rotation=90)

    plt.tight_layout()
    plt.show()


In [4]:
def is_bin_column(col) -> bool:
    # numerische Spaltennamen erlauben
    if isinstance(col, (int, np.integer)):
        return True

    s = str(col)

    # rein numerischer Spaltenname: '0', '1', ...
    if s.isdigit():
        return True
    if re.fullmatch(r"bin_\d+_diff", s):
        return True
    # bin_X oder bin_X_high / bin_X_low
    if re.fullmatch(r"bin_\d+(_high|_low)?", s):
        return True
    if re.fullmatch(r"bin_\d+", s): # 'bin_0', 'bin_1', ...
        return True

    return False

MODELL GP

In [5]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train_gp(xt_train, yt_train, training_iterations=100):
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(xt_train, yt_train, likelihood)

    model.train()
    likelihood.train()

    optimizer = torch.optim.Adam(model.parameters(), lr=0.2)
    mll = ExactMarginalLogLikelihood(likelihood, model)

    losses = []

    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(xt_train)
        loss = -mll(output, yt_train)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

    return model.eval(), likelihood.eval(), losses

In [6]:
def datenfilternundfertigmachenfür(data, temp1, temp2, pres1, pres2):
    # wc(p, T) -> high
    data_high = data[(data.temperature_kelvin == temp1) & (data.pressure_bar == pres1)]
    data_high = data_high.drop_duplicates(subset=["structure_name", "temperature_kelvin", "pressure_bar"])
    feature_columns_high = [col for col in data_high.columns if is_bin_column(col)]
    
    # wc(p, T) -> low
    data_low = data[(data.temperature_kelvin == temp2) & (data.pressure_bar == pres2)]
    data_low = data_low.drop_duplicates(subset=["structure_name", "temperature_kelvin", "pressure_bar"])
    feature_columns_low = [col for col in data_low.columns if is_bin_column(col)]
    
    add_features = False
    diffi = False
    
    data_high["beladung_pro_vol"] = (
        data_high["beladung_atoms"]
        .div(data_high["density_bulk"], axis=0)
        .div(data_high["volume_kubAng"], axis=0)
    )
    
    data_low["beladung_pro_vol"] = (
        data_low["beladung_atoms"]
        .div(data_low["density_bulk"], axis=0)
        .div(data_low["volume_kubAng"], axis=0)
    )
    
    data_high[feature_columns_high] = (
        data_high[feature_columns_high]
        .multiply(data_high["grid.dv"], axis=0)
        .div(data_high["volume_kubAng"], axis=0)
    )
    
    data_low[feature_columns_low] = (
        data_low[feature_columns_low]
        .multiply(data_low["grid.dv"], axis=0)
        .div(data_low["volume_kubAng"], axis=0)
    )
    
    additional_features = ["delta_p", "delta_T", "delta_chem"]
    additional_features12 = ["chem_potential_bulk_high", "chem_potential_bulk_low", "pressure_bar_high", "pressure_bar_low", "temperature_kelvin_high", "temperature_kelvin_low"]
    additional_features12 = ["pressure_bar_high", "pressure_bar_low", "temperature_kelvin_high", "temperature_kelvin_low"]

    merged = pd.merge(
        #data_high[["structure_name", "beladung_pro_vol"]],
        #data_low[["structure_name", "beladung_pro_vol"]],
        data_high,#[cols],
        data_low,#[cols],
        on="structure_name",
        suffixes=("_high", "_low")
    )
    
    # --- Differenzen berechnen (high - low) ---
    if diffi:
        # 1. Alle bin-Spalten mit _high finden (per is_bin_column)
        bin_high_cols = [
            col for col in merged.columns
            if col.endswith("_high") and is_bin_column(col)
        ]
    
        # 2. Für jede dieser Spalten das passende _low finden und die Differenz berechnen
        for col_high in bin_high_cols:
            base = col_high.replace("_high", "")      # z.B. 'bin_0_high' -> 'bin_0'
            col_low = col_high.replace("_high", "_low")
    
            merged[base + "_diff"] = (
                pd.to_numeric(merged[col_high], errors="coerce")
                - pd.to_numeric(merged[col_low], errors="coerce")
            )
    
        # 3. Alte _high/_low-Bin-Spalten löschen
        cols_to_drop = []
        for col_high in bin_high_cols:
            base = col_high.replace("_high", "")
            cols_to_drop.append(base + "_high")
            cols_to_drop.append(base + "_low")
    
        merged = merged.drop(columns=cols_to_drop)

    # wc aus den beiden Zuständen high/low
    merged["working_capacity"] = (merged["beladung_pro_vol_high"] - merged["beladung_pro_vol_low"]).abs()
    merged["delta_T"] = (merged["temperature_kelvin_high"] - merged["temperature_kelvin_low"]).abs()
    merged["delta_p"] = (merged["pressure_bar_high"] - merged["pressure_bar_low"]).abs()
    merged["delta_chem"] = (merged["chem_potential_bulk_high"] - merged["chem_potential_bulk_low"]).abs()

    feature_columns = [col for col in merged.columns if is_bin_column(col)]
    
    if add_features:
        feature_columns += additional_features
    
    merged["working_capacity"] = pd.to_numeric(merged["working_capacity"], errors="coerce")
    
    return merged, feature_columns

NORMALIZE

In [7]:
normalize_feature = True
normalize_labels = True

FOLD - TRAINING - PREDICTION

In [8]:
def GPGPGP123(merged, feature_columns):
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    label = "working_capacity"
    #X = data[feature_columns].values 
    X = merged[feature_columns].values
    #y = data[label].values 
    y = merged[label].values 
    #print(merged.head())
    ids = data.index.values
    
    split_info = []
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(X), start=1):
        x_train = torch.tensor(X[train_idx], dtype=torch.float64)
        y_train = torch.tensor(y[train_idx], dtype=torch.float64)
        x_test = torch.tensor(X[test_idx], dtype=torch.float64)
        y_test = torch.tensor(y[test_idx], dtype=torch.float64)
    
        train_ids = ids[train_idx]
        test_ids = ids[test_idx]
    
        #test_df = data.iloc[test_idx].copy()
        test_df = merged.iloc[test_idx].copy()
        test_df["fold"] = fold
    
        if normalize_feature:
            feature_transformer = MinMaxScaler()
            feature_transformer.fit(x_train)
            xt_train = torch.tensor(feature_transformer.transform(x_train), dtype=torch.float64)
            xt_test = torch.tensor(feature_transformer.transform(x_test), dtype=torch.float64) #*2
        else:
            xt_train = x_train
            xt_test = x_test
    
        # Label-Normalisierung
        if normalize_labels:
            label_transformer = MinMaxScaler()  # oder StandardScaler()
            label_transformer.fit(y_train.unsqueeze(1))
            yt_train = torch.tensor(label_transformer.transform(y_train.unsqueeze(1)).flatten(), dtype=torch.float64)
            yt_test = torch.tensor(label_transformer.transform(y_test.unsqueeze(1)).flatten(), dtype=torch.float64)
        else:
            yt_train = y_train
            yt_test = y_test
    
        # Training
        model, likelihood, losses = train_gp(xt_train, yt_train, training_iterations=200)
    
        # Prediction
        with torch.no_grad():
            prediction = model(xt_test)
            inverse_transformed_prediction = label_transformer.inverse_transform(
                prediction.mean.unsqueeze(1)
            ).squeeze()
            inverse_transformed_prediction = np.where(
                inverse_transformed_prediction > 0, inverse_transformed_prediction, 0
            )
    
        # Ergebnisse
        test_df[f"{label}_pred"] = inverse_transformed_prediction
        test_df["abs_rel_deviation"] = np.abs(
            (test_df[label] - test_df[f"{label}_pred"]) / test_df[label] * 100
        )
    
        split_info.append(test_df)
    
    results = pd.concat(split_info, ignore_index=True)
    return results

In [9]:
def GPGPGP(merged, feature_columns):
    # Falls zu wenig Daten vorhanden sind → direkt überspringen
    if merged.shape[0] < 2 or len(feature_columns) == 0:
        print(f"⚠️  Skip: Zu wenige Daten (n={merged.shape[0]}, features={len(feature_columns)})")
        return pd.DataFrame({})  # leeres DF zurückgeben

    try:
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        label = "working_capacity"
        X = merged[feature_columns].values
        y = merged[label].values
        ids = merged.index.values

        split_info = []

        for fold, (train_idx, test_idx) in enumerate(kf.split(X), start=1):
            x_train = torch.tensor(X[train_idx], dtype=torch.float64)
            y_train = torch.tensor(y[train_idx], dtype=torch.float64)
            x_test = torch.tensor(X[test_idx], dtype=torch.float64)
            y_test = torch.tensor(y[test_idx], dtype=torch.float64)

            test_df = merged.iloc[test_idx].copy()
            test_df["fold"] = fold

            if normalize_feature:
                feature_transformer = MinMaxScaler()
                feature_transformer.fit(x_train)
                xt_train = torch.tensor(feature_transformer.transform(x_train), dtype=torch.float64)
                xt_test = torch.tensor(feature_transformer.transform(x_test), dtype=torch.float64)
            else:
                xt_train = x_train
                xt_test = x_test

            if normalize_labels:
                label_transformer = MinMaxScaler()
                label_transformer.fit(y_train.unsqueeze(1))
                yt_train = torch.tensor(label_transformer.transform(y_train.unsqueeze(1)).flatten(), dtype=torch.float64)
                yt_test = torch.tensor(label_transformer.transform(y_test.unsqueeze(1)).flatten(), dtype=torch.float64)
            else:
                yt_train = y_train
                yt_test = y_test

            model, likelihood, losses = train_gp(xt_train, yt_train, training_iterations=200)

            with torch.no_grad():
                prediction = model(xt_test)
                if normalize_labels:
                    inverse_transformed_prediction = label_transformer.inverse_transform(
                        prediction.mean.unsqueeze(1)
                    ).squeeze()
                else:
                    inverse_transformed_prediction = prediction.mean

                inverse_transformed_prediction = np.where(
                    inverse_transformed_prediction > 0, inverse_transformed_prediction, 0
                )

            test_df[f"{label}_pred"] = inverse_transformed_prediction
            test_df["abs_rel_deviation"] = np.abs(
                (test_df[label] - test_df[f"{label}_pred"]) / test_df[label] * 100
            )

            split_info.append(test_df)

        results = pd.concat(split_info, ignore_index=True)
        return results

    except Exception as e:
        print(f"⚠️  Fehler in GPGPGP → {e}")
        return pd.DataFrame({})  # leeres DF bei Fehler


ALLE DATEN

In [10]:
def printmirmeinenscheiss(results, label="working_capacity"):
    print(f"R²                        : {r2_score(results[label], results[f'{label}_pred']):.4f}")
    print(f"Median APE                : {results['abs_rel_deviation'].median():.2f}%")
    print(f"Mean APE                  : {results['abs_rel_deviation'].mean():.2f}%")
    #print(f"Final Loss                : {losses[-1]:.4f}")
    
    count = (results['abs_rel_deviation'] > 20).sum()
    print(f"Abs rel dev > 20%         : {count} out of {len(results)}")
    print(f"Max abs rel dev           : {results['abs_rel_deviation'].max():.2f}%")


BAYESIAN OPTIMIZATION

In [11]:
from torch.distributions import Normal

def af_log_expIm(mean, var, best_f, xi=0.01):
    """Logarithmic Expected Improvement acquisition function."""

    std = torch.sqrt(var)
    std_safe = torch.clamp(std, min=1e-9)  # Avoid division by zero
    z = (mean - best_f - xi) / std_safe
    normal = Normal(torch.zeros_like(z), torch.ones_like(z))
    cdf = normal.cdf(z)
    pdf = torch.exp(normal.log_prob(z))

    ei = std * (z * cdf + pdf)

    ei_safe = torch.clamp(ei, min=1e-9)  # Avoid log(0)
    log_ei = torch.log(ei_safe)
    return log_ei

In [12]:
def bimbamBO_minimal(
    merged, feature_columns, label="working_capacity",
    train_gp=None, max_iters=100, patience=10, n_initial=1
):
    assert train_gp is not None, "Bitte train_gp=... übergeben."
    candidates = merged.copy().reset_index(drop=True)
    n_candidates = int(len(candidates))

    # globaler Bestwert (Baseline)
    global_best_value = float(candidates[label].max()) if n_candidates else float("nan")

    if n_candidates <= 1:
        return dict(
            n_candidates=n_candidates,
            n_iterations=0,
            best_value=float(candidates[label].max()) if n_candidates else float("nan"),
            global_best_value=global_best_value,
            found_global_best=(n_candidates == 1),
        )

    # Initialauswahl
    n_initial = max(1, min(n_initial, n_candidates-1))
    initial_indices = candidates.nsmallest(n_initial, label).index  # oder random
    selected = candidates.loc[initial_indices].copy()
    candidates = candidates.drop(initial_indices).reset_index(drop=True)

    best_history = [float(selected[label].max())]
    n_iterations = 0

    for i in range(max_iters):
        # Early stopping: keine Änderung der letzten `patience` Einträge
        if len(best_history) >= patience and len(np.unique(best_history[-patience:])) == 1:
            break

        feat_scaler = MinMaxScaler()
        lbl_scaler  = MinMaxScaler()
        train_x = torch.tensor(feat_scaler.fit_transform(selected[feature_columns].values), dtype=torch.float32)
        train_y = torch.tensor(lbl_scaler.fit_transform(selected[[label]].values).flatten(), dtype=torch.float32)
        test_x  = torch.tensor(feat_scaler.transform(candidates[feature_columns].values), dtype=torch.float32)

        model, likelihood, _ = train_gp(train_x, train_y, 250)
        with torch.no_grad():
            pred = model(test_x)
            mean, var = pred.mean, pred.variance

        best_f = train_y.max()
        log_ei = af_log_expIm(mean, var, best_f, 0.01 * best_f)
        index = int(torch.argmax(log_ei).item())

        # move chosen from candidates -> selected
        chosen = candidates.iloc[[index]].copy()
        selected = pd.concat([selected, chosen], ignore_index=True)
        candidates = candidates.drop(candidates.index[index]).reset_index(drop=True)

        best_history.append(float(selected[label].max()))
        n_iterations += 1

        if len(candidates) == 0:
            break

    best_value = float(selected[label].max()) if len(selected) else float("nan")
    found_global_best = (not np.isnan(best_value)) and (not np.isnan(global_best_value)) and (abs(best_value - global_best_value) < 1e-12)

    return dict(
        n_candidates=n_candidates,
        n_iterations=n_iterations,
        best_value=best_value,
        global_best_value=global_best_value,
        found_global_best=found_global_best,
    )

In [13]:
def tempunddruckunique(data):
    temperatures = sorted(data['temperature_kelvin'].unique())
    pressures = sorted(data['pressure_bar'].unique())
    combinations = list(
        data[['temperature_kelvin', 'pressure_bar']]
        .drop_duplicates()
        .itertuples(index=False, name=None)
    )
    return temperatures, pressures, combinations

In [14]:
# dft Daten - beladungen, grid etc
dft_data1 = pd.read_csv('/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/DFT_Data_clean_06_10.csv')
dft_data2 = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/dft_data_temp_pressure_new.csv")
dft_data2["density_bulk"] = (
    dft_data2["density_bulk"]
    .astype(str)                            # sicherstellen, dass alles string ist
    .str.strip()                             # Leerzeichen weg
    .str.replace('[', '', regex=False)       # "[" entfernen
    .str.replace(']', '', regex=False)       # "]" entfernen
)

dft_data2["density_bulk"] = pd.to_numeric(dft_data2["density_bulk"], errors="coerce")
dft_data_all = pd.concat([dft_data1, dft_data2], ignore_index=True).drop_duplicates()

# nur sinnvoller Bereich:
dft_data_all = dft_data_all[
    (dft_data_all['temperature_kelvin'] < 550) &
    (dft_data_all['pressure_bar'] <= 50)
].reset_index(drop=True)

# Feature Daten - bins zu Vext, Vext+chem_res etc
#expV_data = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vext_allTEMP_hist_no_pressure_no_chem.csv")
expV_data = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/Vextchem_allTEMP_pressure_20b_rangeinc.csv")

# Chem_res_bulk explizit - für additional feature 
chem_res = pd.read_csv("/Users/danielbock/MASTERTHESIS/MASTA/DataArchiv/bulk_potentials.csv")

# Kombi aus obigem 
data = pd.merge(dft_data_all, expV_data, 'inner', on=["structure_name", "temperature_kelvin", "pressure_bar"])
data = pd.merge(data, chem_res, 'inner', on=["structure_name", "temperature_kelvin", "pressure_bar"])
#feature_columns = [col for col in data.columns if is_bin_column(col)]
data = data[data.beladung_mol_per_kg > 0]

In [ ]:
# -------- einmalig oben im Skript --------
logger = GPDataLogger(log_dir="logs")

label = "working_capacity"
bo_log_path = "logs/bo_runs_data_tempswing_Vextchem_diff_addfeatchempT_densB.csv"  # eine Datei, eine Zeile pro (T,p1,p2)

temperatures, pressures, combis = tempunddruckunique(data)

results_gp = []
results_bo = []
counter_bad = 0
counter_good = 0

for p in pressures:
    sub = data[data['pressure_bar'] == p]
    sub = sub.drop_duplicates(subset=['temperature_kelvin', 'structure_name'], keep='first').reset_index(drop=True)
    print(sub.shape)

    for T1 in temperatures:
        for T2 in temperatures:
            if T1 == T2:
                continue

            p1 = p
            p2 = p
            # --- Datenaufbereitung ---
            merged, feature_columns = datenfilternundfertigmachenfür(sub, T1, T2, p1, p2)

            # Problemfall-Shape optional protokollieren und überspringen
            if merged.shape == (245, 88):
                counter_bad += 1
                logger.log_summary(
                    T1=T1, T2=T2, p1=p1, p2=p2,
                    results_df=pd.DataFrame({label: [], f"{label}_pred": [], "abs_rel_deviation": []}),
                    extra_info={"note": "skip_due_to_bad_shape", "shape": list(merged.shape)}
                )
                continue

            # --- GP-Run (deine bestehende Funktion) ---
            results = GPGPGP(merged, feature_columns)
            
            if results is None or results.empty:
                logger.log_summary(
                    T1=T1, T2=T2, p1=p1, p2=p2,
                    results_df=pd.DataFrame({label: [], f"{label}_pred": [], "abs_rel_deviation": []}),
                    label=label,
                    extra_info={"note": "skip_due_to_invalid_data"}
                )
                continue
            
            # --- Wenn alles okay, normal loggen ---
            logger.log_summary(
                T1=T1, T2=T2, p1=p1, p2=p2,
                results_df=results,
                label=label,
            )
            logger.log_predictions(T1=T1, T2=T2, p1=p1, p2=p2, results_df=results, label=label)

            print(f"Zustand --> {p}bar, {T1}kelvin und {T2}kelvin")
            #printmirmeinenscheiss(results, label=label)

            counter_good += 1
            results_gp.append({"T1": T1, "T2": T2, "p1": p1, "p2": p2})

            # --- BO-Run + Logging für jede Variation ---
            bo_metrics = bimbamBO_minimal(
                merged=merged,
                feature_columns=feature_columns,
                label="working_capacity",
                train_gp=train_gp,         # deine Train-Funktion
                max_iters=100,
                patience=10,
                n_initial=1
            )

            append_log_row(bo_log_path, {
                "timestamp": datetime.now().isoformat(timespec="seconds"),
                "temperature_kelvin_T1": T1,
                "temperature_kelvin_T2": T2,
                "pressure_bar_1": p1,
                "pressure_bar_2": p2,
                "n_candidates": bo_metrics["n_candidates"],
                "n_iterations": bo_metrics["n_iterations"],
                "best_value": bo_metrics["best_value"],
                "global_best_value": bo_metrics["global_best_value"],
                "found_global_best": bo_metrics["found_global_best"],
            })

In [15]:
tmwp swing  T1 - 300 T2 - 400 - 1bar
pressure swing p1 - p1 - 1 p2 - 0.01 p3 - 5 temp 300
---> Siehe Literatur

(2450, 34)
Zustand --> 0.1bar, 300.0kelvin und 325.0kelvin


/Users/danielbock/MASTERTHESIS/MASTA/.venv/lib/python3.11/site-packages/gpytorch/distributions/multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(


Zustand --> 0.1bar, 300.0kelvin und 350.0kelvin


/Users/danielbock/MASTERTHESIS/MASTA/.venv/lib/python3.11/site-packages/gpytorch/distributions/multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(


Zustand --> 0.1bar, 300.0kelvin und 375.0kelvin


...

Zustand --> 50.0bar, 500.0kelvin und 450.0kelvin


/Users/danielbock/MASTERTHESIS/MASTA/.venv/lib/python3.11/site-packages/gpytorch/distributions/multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(


Zustand --> 50.0bar, 500.0kelvin und 475.0kelvin


/Users/danielbock/MASTERTHESIS/MASTA/.venv/lib/python3.11/site-packages/gpytorch/distributions/multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(


⚠️  Skip: Zu wenige Daten (n=0, features=23)
⚠️  Skip: Zu wenige Daten (n=0, features=23)
⚠️  Skip: Zu wenige Daten (n=0, features=23)
⚠️  Skip: Zu wenige Daten (n=0, features=23)
⚠️  Skip: Zu wenige Daten (n=0, features=23)
⚠️  Skip: Zu wenige Daten (n=0, features=23)
⚠️  Skip: Zu wenige Daten (n=0, features=23)
⚠️  Skip: Zu wenige Daten (n=0, features=23)
⚠️  Skip: Zu wenige Daten (n=0, features=23)
⚠️  Skip: Zu wenige Daten (n=0, features=23)


In [16]:
temperatures, pressures, combis = tempunddruckunique(data)

results_gp = []
results_bo = []
counter_bad = 0
counter_good = 0
for T in temperatures:
    sub = data[data['temperature_kelvin']==T]
    sub = sub.drop_duplicates(subset=['pressure_bar', 'structure_name'], keep='first').reset_index(drop=True)
    print(sub.shape)
    for p1 in pressures:
        for p2 in pressures:
            if p1 == p2:
                continue
            # Hier muss der shit rein
            merged, feature_columns = datenfilternundfertigmachenfür(sub, T, T, p1, p2) # funktioniert
            print(merged)

(1715, 34)
    structure_name  pressure_bar_high  temperature_kelvin_high  \
0              EON                0.1                    300.0   
1              OKO                0.1                    300.0   
2              MWW                0.1                    300.0   
3              SFS                0.1                    300.0   
4              DDR                0.1                    300.0   
..             ...                ...                      ...   
240            JBW                0.1                    300.0   
241            LTJ                0.1                    300.0   
242            MEI                0.1                    300.0   
243            RTH                0.1                    300.0   
244            WEN                0.1                    300.0   

     volume_kubAng_high  grid.dv_high  density_Atmos_per_kubAng_high  \
0           3563.018109      0.013592                     104.297410   
1           3884.589614      0.014819               